#### Simple Version Counting

The most _straightforward_ way to __track versions__ is to _add an integer column_ to the mapped table, then establish it as the `version_id_col` within the mapper options.

In [1]:
from sqlalchemy import Column, Integer, String
from sqlalchemy.orm import declarative_base

In [2]:
Base = declarative_base()

In [3]:
class User(Base):
    __tablename__ = "user"
    
    id = Column(Integer, primary_key=True)
    version_id = Column(Integer, nullable=False)
    name = Column(String(50), nullable=False)
    
    __mapper_args__ = {"version_id_col": version_id}

> ##### Note
>
> It is __strongly recommended__ that the `version_id` column be made _NOT NULL_. The _versioning feature_ __does not support a NULL value in the versioning column__.

Above, the _User_ mapping __tracks integer versions__ using the column `version_id`. When an object of type _User_ is _first flushed_, the `version_id` column will be given a value of `"1"`. Then, an _UPDATE_ of the table later on __will always be emitted__ in a manner similar to the following:

```
UPDATE user SET version_id=:version_id, name=:name
WHERE user.id = :user_id AND user.version_id = :user_version_id
{"name": "new name", "version_id": 2, "user_id": 1, "user_version_id": 1}
```

The above _UPDATE_ statement is updating the row that not only matches `user.id = 1`, it also is __requiring__ that `user.version_id = 1`, where `"1"` is the __last version identifier__ we've been known to use on this object. If a transaction elsewhere has modified the row independently, this _version id_ will no longer match, and the _UPDATE_ statement will __report that no rows matched__; this is the condition that SQLAlchemy tests, that __exactly one row matched our UPDATE (or DELETE) statement__. If _zero rows match_, that _indicates_ our version of the data is __stale__, and a `StaleDataError` is raised.